In [ ]:
from wildnlp.aspects.utils import compose, change_model_prediction, change_most_important_word, change_random_word
from wildnlp.aspects import QWERTY, Swap, SwapWords
from wildnlp.datasets import SampleDataset
from wildnlp.aspects.selector import EmbeddingSelector
from wildnlp.aspects.word_creator import HypernymCreator, SynonymCreator
from flair.embeddings import BertEmbeddings

# Basic example

In [ ]:
# Create a dataset object and load the dataset
dataset = SampleDataset()
dataset.load()


# Create a composed corruptor function.
# Functions will be applied in the same order they appear.
composed = compose(Swap(), QWERTY())

# Apply the function to the dataset
modified = dataset.apply(composed)
modified

In [ ]:
# Pass the str as argument
composed("Example sentence to change")

# Swap words example

In [ ]:
from wildnlp.aspects.word_creator import HypernymCreator, SynonymCreator
from wildnlp.aspects.selector import EmbeddingSelector

# Prepare embeddings
embedding =  BertEmbeddings()

# Prepare selector
selector = EmbeddingSelector(embedding)

In [ ]:
# Create SwapWords with hypernyms
composed = compose(SwapWords(word_creator=HypernymCreator(),pos_to_change=['a', 'v', 'r'], selector=selector, 
                             words_percentage=1), )

composed("Be careful, this is very dangerous lion, he can hurt someone")

In [ ]:
# Create SwapWords with synonyms
composed = compose(SwapWords(word_creator=SynonymCreator(),pos_to_change=['a', 'v', 'r'], selector=selector, 
                             words_percentage=1), )

composed("Be careful, this is very dangerous lion, he can hurt someone")

# Additional functions

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

# Load flair sentiment classifier
classifier = TextClassifier.load('en-sentiment')

In [ ]:
from torch import nn
from typing import Callable, List
import numpy as np
from flair.training_utils import clear_embeddings

# Prepare model pipeline
def create_classify_flair(model: nn.Module) -> \
        Callable[[List[str]], np.array]:
   
    def classify(user_input: List[str]) -> np.array:
        mini_batch_size = 32
        model.eval()
        sentences = [Sentence(text) for text in user_input]
        for i, s in enumerate(sentences):
            s.add_label(str(i))
        batches = [sentences[i:i + mini_batch_size] for i in range(0, len(sentences), mini_batch_size)]
        pred = []
        for batch in batches:
            output = model(batch)
            indices = [int(s.get_label_names()[0]) for s in batch]
            pred_batch = output.cpu().data.numpy()
            pred_batch = [(np.exp(p) / np.sum(np.exp(p))) for p in pred_batch] 
            pred_batch = [x for _, x in sorted(zip(indices, pred_batch), key=lambda pair: pair[0])]
            pred += pred_batch
            clear_embeddings(batch)
        proba = np.array(pred)
        return proba
    return classify


pipeline = create_classify_flair(classifier)

In [ ]:
text = 'This is very good option, but have bad consequences'
label = 0

pipeline([text])

# Change most important

In [ ]:
change_most_important_word(text, label, pipeline, QWERTY(), if_percent=False, how_many=1)

# Change model prediction

In [ ]:
change_model_prediction(text, label, pipeline, [QWERTY(), Swap()], verbose=True)

# Change random word

In [ ]:
change_random_word(text, QWERTY(), if_percent=False, how_many=3, seed=23)